In [ ]:

import plotly.graph_objects as go
import numpy as np


# Define ellipsoid parameters (WGS84 model for Earth)
semi_major = 6378.137  # semi-major axis (km)
semi_minor = 6356.752  # semi-minor axis (km)


# Generate points on the ellipsoid surface
u = np.linspace(0, 2 * np.pi, 200)  # Longitude
v = np.linspace(0, np.pi, 100)        # Latitude
x = semi_major * np.outer(np.cos(u), np.sin(v))
y = semi_major * np.outer(np.sin(u), np.sin(v))
z = semi_minor * np.outer(np.ones(np.size(u)), np.cos(v))


# Calculate the Euclidean distance from the center
distance_from_center = np.sqrt(x**2 + y**2 + z**2)

# Normalize the distances
max_distance = np.max(distance_from_center)
normalized_distance = distance_from_center / max_distance


# Define a custom colorscale from light grey to white
custom_colorscale = [
    [0, 'lightgrey'],
    [1, 'white']
]


ellipsoid_surface = go.Surface(
    x=x, 
    y=y, 
    z=z, 
    colorscale='Viridis',  # Use the custom colorscale
    surfacecolor=normalized_distance,  # Set color scale based on normalized distance
    opacity=0.5,
     contours = {
        "x": {"show": True, "start": 0, "end": 1, "size": 1, "color":"red"},
        "y": {"show": True, "start": 0, "end": 1, "size": 1, "color":"green"},
        "z": {"show": True, "start": 0, "end": 1, "size": 1, "color":"blue"},


    },
)

# Create contours every 30 degrees
contour_latitudes = np.arange(-60, 61, 5)  # Latitude from -90 to 90 degrees
contour_longitudes = np.arange(-30, 31, 5)  # Longitude from -180 to 180 degrees

# Create contour lines for latitude
contour_lines_latitude_x = np.outer(np.cos(np.radians(contour_latitudes)), np.cos(np.radians(contour_longitudes))) * semi_major
contour_lines_latitude_y = np.outer(np.cos(np.radians(contour_latitudes)), np.sin(np.radians(contour_longitudes))) * semi_major
contour_lines_latitude_z = np.outer(np.sin(np.radians(contour_latitudes)), np.ones(len(contour_longitudes))) * semi_minor

# Create contour lines for longitude
contour_lines_longitude_x = np.outer(np.cos(np.radians(contour_latitudes)), np.cos(np.radians(contour_longitudes))) * semi_major
contour_lines_longitude_y = np.outer(np.cos(np.radians(contour_latitudes)), np.sin(np.radians(contour_longitudes))) * semi_major
contour_lines_longitude_z = np.outer(np.sin(np.radians(contour_latitudes)), np.ones(len(contour_longitudes))) * semi_minor

# Create contour lines for latitude
contour_lines_latitude_x = []
contour_lines_latitude_y = []
contour_lines_latitude_z = []
for lat in contour_latitudes:
    lat_rad = np.radians(lat)
    x_lat = semi_major * np.cos(lat_rad) * np.cos(np.radians(contour_longitudes))
    y_lat = semi_major * np.cos(lat_rad) * np.sin(np.radians(contour_longitudes))
    z_lat = semi_minor * np.sin(lat_rad) * np.ones_like(contour_longitudes)
    contour_lines_latitude_x.extend(x_lat.tolist() + [None])
    contour_lines_latitude_y.extend(y_lat.tolist() + [None])
    contour_lines_latitude_z.extend(z_lat.tolist() + [None])

# Create contour lines for longitude
contour_lines_longitude_x = []
contour_lines_longitude_y = []
contour_lines_longitude_z = []
for lon in contour_longitudes:
    lon_rad = np.radians(lon)
    x_lon = semi_major * np.cos(np.radians(contour_latitudes)) * np.cos(lon_rad)
    y_lon = semi_major * np.cos(np.radians(contour_latitudes)) * np.sin(lon_rad)
    z_lon = semi_minor * np.sin(np.radians(contour_latitudes))
    contour_lines_longitude_x.extend(x_lon.tolist() + [None])
    contour_lines_longitude_y.extend(y_lon.tolist() + [None])
    contour_lines_longitude_z.extend(z_lon.tolist() + [None])


# Flatten the contour lines arrays
contour_lines_latitude_x = np.array(contour_lines_latitude_x).flatten()
contour_lines_latitude_y = np.array(contour_lines_latitude_y).flatten()
contour_lines_latitude_z = np.array(contour_lines_latitude_z).flatten()
contour_lines_longitude_x = np.array(contour_lines_longitude_x).flatten()
contour_lines_longitude_y = np.array(contour_lines_longitude_y).flatten()
contour_lines_longitude_z = np.array(contour_lines_longitude_z).flatten()

# Create contour traces
contour_latitude_trace = go.Scatter3d(
    x=contour_lines_latitude_x,
    y=contour_lines_latitude_y,
    z=contour_lines_latitude_z,
    mode='lines',
    line=dict(color='black', width=1),
    showlegend=False
)

contour_longitude_trace = go.Scatter3d(
    x=contour_lines_longitude_x,
    y=contour_lines_longitude_y,
    z=contour_lines_longitude_z,
    mode='lines',
    line=dict(color='black', width=1),
    showlegend=False
)


# Set up the layout for the 3D plot
layout = go.Layout(
    title='3D Visualization of the WGS84 Ellipsoid with 5 degree graticule and local tangent plane',
    scene=dict(
        xaxis_title='X (km)',
        yaxis_title='Y (km)',
        zaxis_title='Z (km)',
        aspectmode='data'
    )
)

# Create axis marks at the center
axis_length = max(semi_major, semi_minor) * 1.1  # Extend a bit beyond the ellipsoid
axis_x = go.Scatter3d(
    x=[0, axis_length/5],
    y=[0, 0],
    z=[0, 0],
    mode='lines',
    line=dict(color='red', width=4),
    showlegend=False
)

axis_y = go.Scatter3d(
    x=[0, 0],
    y=[0, axis_length/5],
    z=[0, 0],
    mode='lines',
    line=dict(color='green', width=4),
    showlegend=False
)

axis_z = go.Scatter3d(
    x=[0, 0],
    y=[0, 0],
    z=[0, axis_length/5],
    mode='lines',
    line=dict(color='blue', width=4),
    showlegend=False
)


# Define a surface point in geodetic coordinates (latitude, longitude)
lat = 15  # degrees
lon = 0 # degrees
h = 0     # height above ellipsoid

# Convert lat/lon to radians
lat_rad = np.radians(lat)
lon_rad = np.radians(lon)

# Calculate the prime vertical radius of curvature
N = semi_major / np.sqrt(1 - (1 - (semi_minor / semi_major)**2) * np.sin(lat_rad)**2)

# Convert geodetic coordinates to ECEF
x_p = (N + h) * np.cos(lat_rad) * np.cos(lon_rad)
y_p = (N + h) * np.cos(lat_rad) * np.sin(lon_rad)
z_p = (N * (semi_minor / semi_major)**2 + h) * np.sin(lat_rad)



# Calculate the normal vector at the point
normal_vector = np.array([x_p / semi_major**2, y_p / semi_major**2, z_p / semi_minor**2])
normal_vector /= np.linalg.norm(normal_vector)

# Define the local tangent plane using two orthogonal vectors in the plane
tangent_vector1 = np.cross(normal_vector, [0, 0, 1])
tangent_vector1 /= np.linalg.norm(tangent_vector1)
tangent_vector2 = np.cross(normal_vector, tangent_vector1)

plane_size = 2000  # Size of the plane in km

# Create points on the tangent plane
tangent_plane_x = []
tangent_plane_y = []
tangent_plane_z = []
for i in [-1, 1]:
    for j in [-1, 1]:
        point = np.array([x_p, y_p, z_p]) + i * plane_size * tangent_vector1 + j * plane_size * tangent_vector2
        tangent_plane_x.append(point[0] )
        tangent_plane_y.append(point[1])
        tangent_plane_z.append(point[2])
tangent_plane_x.append(tangent_plane_x[0])
tangent_plane_y.append(tangent_plane_y[0])
tangent_plane_z.append(tangent_plane_z[0])
# Create the tangent plane trace
tangent_plane_trace = go.Mesh3d(
    x=tangent_plane_x,
    y=tangent_plane_y,
    z=tangent_plane_z,
    color='grey',
    opacity=0.3,
    showlegend=True
)

# Add None values to break the lines between the corners
tangent_plane_outline_x = tangent_plane_x + [None, tangent_plane_x[0], tangent_plane_x[1], None, tangent_plane_x[1], tangent_plane_x[3], None, tangent_plane_x[3], tangent_plane_x[2], None, tangent_plane_x[2], tangent_plane_x[0]]
tangent_plane_outline_y = tangent_plane_y + [None, tangent_plane_y[0], tangent_plane_y[1], None, tangent_plane_y[1], tangent_plane_y[3], None, tangent_plane_y[3], tangent_plane_y[2], None, tangent_plane_y[2], tangent_plane_y[0]]
tangent_plane_outline_z = tangent_plane_z + [None, tangent_plane_z[0], tangent_plane_z[1], None, tangent_plane_z[1], tangent_plane_z[3], None, tangent_plane_z[3], tangent_plane_z[2], None, tangent_plane_z[2], tangent_plane_z[0]]


tangent_plane_outline_trace = go.Scatter3d(
    x=tangent_plane_outline_x,
    y=tangent_plane_outline_y,
    z=tangent_plane_outline_z,
    mode='lines',
    line=dict(color='black', width=1),
    showlegend=False
)

# Create the line from the origin to the center of the tangent plane
line_to_center_trace = go.Scatter3d(
    x=[0, x_p],
    y=[0, y_p],
    z=[0, z_p],
    mode='lines',
    line=dict(color='red', width=2),
    showlegend=False
)

# Create the line from the origin to the center of the tangent plane
line_to_0_0 = go.Scatter3d(
    x=[0, semi_major],
    y=[0, 0],
    z=[0, 0],
    mode='lines',
    line=dict(color='red', width=2),
    showlegend=False
)


# Create a figure with the ellipsoid surface and contours
fig = go.Figure(data=[ellipsoid_surface, 
                      contour_latitude_trace, 
                      contour_longitude_trace, 
                      axis_x,
                      axis_y,
                      axis_z,
                      tangent_plane_trace,
                      tangent_plane_outline_trace,
                      line_to_center_trace,
                      line_to_0_0], 

                      layout=layout)

fig.show()



